### RUN IN TERMINAL FOR DATABASE CREATION

docker ps (to see the running containers)

docker stop id (write instead of id what you want to delete)

docker rm -f $(docker ps -aq) (delete every running container)

docker run --name my-postgres-db -e POSTGRES_USER=master -e POSTGRES_PASSWORD=pass -e POSTGRES_DB=GLOBBING -p 5432:5432 -d postgres

# Initialize the database by creating the tables

In [1]:
from zenq.api.prepare_db import db
m=db()
m.main()

2023-05-01 17:05:07,397 - prepare_db.py - INFO - db (prepare_db.py:36)
2023-05-01 17:05:07,397 main db


Initializing the database.. done


# Insert Facts into database

In [1]:
from zenq.api.endpoints import insert_facts
insert_facts('globbing.csv', 'Customer', 'Gender', 'InvoiceId', 'Date', 'Product_weight', 'Product_weight')

Inserting facts for Customer from file globbing.csv
Skipping row with duplicate invoice_id: INV-101
Skipping row with duplicate invoice_id: INV-102
Skipping row with duplicate invoice_id: INV-103
Skipping row with duplicate invoice_id: INV-104
Skipping row with duplicate invoice_id: INV-105
Skipping row with duplicate invoice_id: INV-106
Skipping row with duplicate invoice_id: INV-107
Skipping row with duplicate invoice_id: INV-108
Skipping row with duplicate invoice_id: INV-109
Skipping row with duplicate invoice_id: INV-110
Skipping row with duplicate invoice_id: INV-111
Skipping row with duplicate invoice_id: INV-112
Skipping row with duplicate invoice_id: INV-113
Skipping row with duplicate invoice_id: INV-114
Skipping row with duplicate invoice_id: INV-115
Skipping row with duplicate invoice_id: INV-116
Skipping row with duplicate invoice_id: INV-117
Skipping row with duplicate invoice_id: INV-118
Skipping row with duplicate invoice_id: INV-119
Skipping row with duplicate invoice_

Skipping row with duplicate invoice_id: INV-310
Skipping row with duplicate invoice_id: INV-311
Skipping row with duplicate invoice_id: INV-312
Skipping row with duplicate invoice_id: INV-313
Skipping row with duplicate invoice_id: INV-314
Skipping row with duplicate invoice_id: INV-315
Skipping row with duplicate invoice_id: INV-316
Skipping row with duplicate invoice_id: INV-317
Skipping row with duplicate invoice_id: INV-318
Skipping row with duplicate invoice_id: INV-319
Skipping row with duplicate invoice_id: INV-320
Skipping row with duplicate invoice_id: INV-321
Skipping row with duplicate invoice_id: INV-322
Skipping row with duplicate invoice_id: INV-323
Skipping row with duplicate invoice_id: INV-324
Skipping row with duplicate invoice_id: INV-325
Skipping row with duplicate invoice_id: INV-326
Skipping row with duplicate invoice_id: INV-327
Skipping row with duplicate invoice_id: INV-328
Skipping row with duplicate invoice_id: INV-329
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-538
Skipping row with duplicate invoice_id: INV-539
Skipping row with duplicate invoice_id: INV-540
Skipping row with duplicate invoice_id: INV-541
Skipping row with duplicate invoice_id: INV-542
Skipping row with duplicate invoice_id: INV-543
Skipping row with duplicate invoice_id: INV-544
Skipping row with duplicate invoice_id: INV-545
Skipping row with duplicate invoice_id: INV-546
Skipping row with duplicate invoice_id: INV-547
Skipping row with duplicate invoice_id: INV-548
Skipping row with duplicate invoice_id: INV-549
Skipping row with duplicate invoice_id: INV-550
Skipping row with duplicate invoice_id: INV-551
Skipping row with duplicate invoice_id: INV-552
Skipping row with duplicate invoice_id: INV-553
Skipping row with duplicate invoice_id: INV-554
Skipping row with duplicate invoice_id: INV-555
Skipping row with duplicate invoice_id: INV-556
Skipping row with duplicate invoice_id: INV-557
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-720
Skipping row with duplicate invoice_id: INV-721
Skipping row with duplicate invoice_id: INV-722
Skipping row with duplicate invoice_id: INV-723
Skipping row with duplicate invoice_id: INV-724
Skipping row with duplicate invoice_id: INV-725
Skipping row with duplicate invoice_id: INV-726
Skipping row with duplicate invoice_id: INV-727
Skipping row with duplicate invoice_id: INV-728
Skipping row with duplicate invoice_id: INV-729
Skipping row with duplicate invoice_id: INV-730
Skipping row with duplicate invoice_id: INV-731
Skipping row with duplicate invoice_id: INV-732
Skipping row with duplicate invoice_id: INV-733
Skipping row with duplicate invoice_id: INV-734
Skipping row with duplicate invoice_id: INV-735
Skipping row with duplicate invoice_id: INV-736
Skipping row with duplicate invoice_id: INV-737
Skipping row with duplicate invoice_id: INV-738
Skipping row with duplicate invoice_id: INV-739
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-912
Skipping row with duplicate invoice_id: INV-913
Skipping row with duplicate invoice_id: INV-914
Skipping row with duplicate invoice_id: INV-915
Skipping row with duplicate invoice_id: INV-916
Skipping row with duplicate invoice_id: INV-917
Skipping row with duplicate invoice_id: INV-918
Skipping row with duplicate invoice_id: INV-919
Skipping row with duplicate invoice_id: INV-920
Skipping row with duplicate invoice_id: INV-921
Skipping row with duplicate invoice_id: INV-922
Skipping row with duplicate invoice_id: INV-923
Skipping row with duplicate invoice_id: INV-924
Skipping row with duplicate invoice_id: INV-925
Skipping row with duplicate invoice_id: INV-926
Skipping row with duplicate invoice_id: INV-927
Skipping row with duplicate invoice_id: INV-928
Skipping row with duplicate invoice_id: INV-929
Skipping row with duplicate invoice_id: INV-930
Skipping row with duplicate invoice_id: INV-931
Skipping row with duplicate invoice_id: 

...

Skipping row with duplicate invoice_id: INV-21483
Skipping row with duplicate invoice_id: INV-21484
Skipping row with duplicate invoice_id: INV-21485
Skipping row with duplicate invoice_id: INV-21486
Skipping row with duplicate invoice_id: INV-21487
Skipping row with duplicate invoice_id: INV-21488
Skipping row with duplicate invoice_id: INV-21489
Skipping row with duplicate invoice_id: INV-21490
Skipping row with duplicate invoice_id: INV-21491
Skipping row with duplicate invoice_id: INV-21492
Skipping row with duplicate invoice_id: INV-21493
Skipping row with duplicate invoice_id: INV-21494
Skipping row with duplicate invoice_id: INV-21495
Skipping row with duplicate invoice_id: INV-21496
Skipping row with duplicate invoice_id: INV-21497
Skipping row with duplicate invoice_id: INV-21498
Skipping row with duplicate invoice_id: INV-21499
Skipping row with duplicate invoice_id: INV-21500
Skipping row with duplicate invoice_id: INV-21501
Skipping row with duplicate invoice_id: INV-21502


Skipping row with duplicate invoice_id: INV-21689
Skipping row with duplicate invoice_id: INV-21690
Skipping row with duplicate invoice_id: INV-21691
Skipping row with duplicate invoice_id: INV-21692
Skipping row with duplicate invoice_id: INV-21693
Skipping row with duplicate invoice_id: INV-21694
Skipping row with duplicate invoice_id: INV-21695
Skipping row with duplicate invoice_id: INV-21696
Skipping row with duplicate invoice_id: INV-21697
Skipping row with duplicate invoice_id: INV-21698
Skipping row with duplicate invoice_id: INV-21699
Skipping row with duplicate invoice_id: INV-21700
Skipping row with duplicate invoice_id: INV-21701
Skipping row with duplicate invoice_id: INV-21702
Skipping row with duplicate invoice_id: INV-21703
Skipping row with duplicate invoice_id: INV-21704
Skipping row with duplicate invoice_id: INV-21705
Skipping row with duplicate invoice_id: INV-21706
Skipping row with duplicate invoice_id: INV-21707
Skipping row with duplicate invoice_id: INV-21708


Skipping row with duplicate invoice_id: INV-21885
Skipping row with duplicate invoice_id: INV-21886
Skipping row with duplicate invoice_id: INV-21887
Skipping row with duplicate invoice_id: INV-21888
Skipping row with duplicate invoice_id: INV-21889
Skipping row with duplicate invoice_id: INV-21890
Skipping row with duplicate invoice_id: INV-21891
Skipping row with duplicate invoice_id: INV-21892
Skipping row with duplicate invoice_id: INV-21893
Skipping row with duplicate invoice_id: INV-21894
Skipping row with duplicate invoice_id: INV-21895
Skipping row with duplicate invoice_id: INV-21896
Skipping row with duplicate invoice_id: INV-21897
Skipping row with duplicate invoice_id: INV-21898
Skipping row with duplicate invoice_id: INV-21899
Skipping row with duplicate invoice_id: INV-21900
Skipping row with duplicate invoice_id: INV-21901
Skipping row with duplicate invoice_id: INV-21902
Skipping row with duplicate invoice_id: INV-21903
Skipping row with duplicate invoice_id: INV-21904


2023-05-01 17:56:02,203 - endpoints.py - ERROR - insert_facts (endpoints.py:119)
2023-05-01 17:56:02,203 insert_facts insert_facts
2023-05-01 17:56:02,205 - endpoints.py - WARNING - insert_facts (endpoints.py:120)
2023-05-01 17:56:02,205 insert_facts insert_facts
2023-05-01 17:56:02,207 - endpoints.py - INFO - insert_facts (endpoints.py:121)
2023-05-01 17:56:02,207 insert_facts insert_facts


Skipping row with duplicate invoice_id: INV-22069
Skipping row with duplicate invoice_id: INV-22070
Skipping row with duplicate invoice_id: INV-22071
Skipping row with duplicate invoice_id: INV-22072
Skipping row with duplicate invoice_id: INV-22073
Skipping row with duplicate invoice_id: INV-22074
Skipping row with duplicate invoice_id: INV-22075
Skipping row with duplicate invoice_id: INV-22076
Skipping row with duplicate invoice_id: INV-22077
Skipping row with duplicate invoice_id: INV-22078
Skipping row with duplicate invoice_id: INV-22079
Skipping row with duplicate invoice_id: INV-22080
Skipping row with duplicate invoice_id: INV-22081
Skipping row with duplicate invoice_id: INV-22082
Skipping row with duplicate invoice_id: INV-22083
Skipping row with duplicate invoice_id: INV-22084
Skipping row with duplicate invoice_id: INV-22085
Skipping row with duplicate invoice_id: INV-22086
Skipping row with duplicate invoice_id: INV-22087
Skipping row with duplicate invoice_id: INV-22088


# Define the Model and input data in result schema of database

In [2]:
from zenq.clvmodels.pareto import Model
model = Model()

## Compute key metrics for CLV

In [3]:
cltv = model.cltv_df()
cltv

2023-05-01 17:56:28,445 _init_num_threads NumExpr defaulting to 8 threads.


,customer_id,min_date,recency,T,frequency,monetary
0,KVO444312,2022-09-13,163,230,14,62.43
1,VBV804469,2022-09-04,200,239,14,58.16
2,BAE240832,2022-09-13,196,230,3,8.97
3,AOK457989,2022-09-14,189,229,5,25.09
4,XZF813575,2022-09-05,179,238,11,42.98
...,...,...,...,...,...,...
1956,DXT747575,2022-09-12,206,231,7,28.18
1957,VOZ592840,2023-01-14,52,107,2,9.99
1958,ZCQ782298,2022-09-13,207,230,20,79.30
1959,YNW866090,2022-09-02,143,241,11,47.75


## Categorization of customers based on RFM scores

In [5]:
rfm = model.rfm()
rfm

,customer_id,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,KVO444312,4,4,4,44,LOYAL CUSTOMER
1,VBV804469,2,4,4,24,AT RISK
2,BAE240832,2,1,1,21,HIBERNATING
3,AOK457989,3,1,2,31,ABOUT TO SLEEP
4,XZF813575,3,3,3,33,NEED ATTENTION
...,...,...,...,...,...,...
1956,DXT747575,1,2,2,12,HIBERNATING
1957,VOZ592840,5,1,1,51,NEW CUSTOMERS
1958,ZCQ782298,1,5,5,15,CANT LOSE
1959,YNW866090,4,3,3,43,POTENTIAL LOYALIST


## Fit into Pareto model

In [4]:
fit = model.fit_paretonbd()
fit

/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


<lifetimes.ParetoNBDFitter: fitted with 1960 subjects, alpha: 132.40, beta: 359591.69, r: 7.18, s: 369.32>

## Model parameters

In [5]:
parameters = model.model_params
parameters

<bound method Model.model_params of <zenq.clvmodels.pareto.Model object at 0x7f7c30c11790>>

## Predictions for 30,90,180,360 days

In [10]:
pareto = model.predict_paretonbd
pareto

<bound method Model.predict_paretonbd of <zenq.clvmodels.pareto.Model object at 0x7fc178ad9c10>>

## Customer aliveness

In [11]:
alive = model.customer_is_alive
alive

<bound method Model.customer_is_alive of <zenq.clvmodels.pareto.Model object at 0x7fc178ad9c10>>

In [ ]:
# from lifetimes import ParetoNBDFitter 
from lifetimes.utils import summary_data_from_transaction_data
import pandas as pd 
from zenq.clvmodels.pareto import Model
model = Model()
result = model.fit_paretonbd()
result = model

In [1]:
from zenq.api.prepare_db import db

Enter your database credentials
Username: aua
Password: mysecretpassword
Database name: GLOBBING


In [2]:
from zenq.api.endpoints import insert_facts
a = insert_facts('globbing.csv', 'Customer', 'Gender', 'InvoiceId', 'Date', 'Product_weight', 'Product_price')

Inserting facts for customer Customer from file globbing.csv
Skipping row with duplicate invoice_id: INV-101
Skipping row with duplicate invoice_id: INV-102
Skipping row with duplicate invoice_id: INV-103
Skipping row with duplicate invoice_id: INV-104
Skipping row with duplicate invoice_id: INV-105
Skipping row with duplicate invoice_id: INV-106
Skipping row with duplicate invoice_id: INV-107
Skipping row with duplicate invoice_id: INV-108
Skipping row with duplicate invoice_id: INV-109
Skipping row with duplicate invoice_id: INV-110
Skipping row with duplicate invoice_id: INV-111
Skipping row with duplicate invoice_id: INV-112
Skipping row with duplicate invoice_id: INV-113
Skipping row with duplicate invoice_id: INV-114
Skipping row with duplicate invoice_id: INV-115
Skipping row with duplicate invoice_id: INV-116
Skipping row with duplicate invoice_id: INV-117
Skipping row with duplicate invoice_id: INV-118
Skipping row with duplicate invoice_id: INV-119
Skipping row with duplicate

Skipping row with duplicate invoice_id: INV-308
Skipping row with duplicate invoice_id: INV-309
Skipping row with duplicate invoice_id: INV-310
Skipping row with duplicate invoice_id: INV-311
Skipping row with duplicate invoice_id: INV-312
Skipping row with duplicate invoice_id: INV-313
Skipping row with duplicate invoice_id: INV-314
Skipping row with duplicate invoice_id: INV-315
Skipping row with duplicate invoice_id: INV-316
Skipping row with duplicate invoice_id: INV-317
Skipping row with duplicate invoice_id: INV-318
Skipping row with duplicate invoice_id: INV-319
Skipping row with duplicate invoice_id: INV-320
Skipping row with duplicate invoice_id: INV-321
Skipping row with duplicate invoice_id: INV-322
Skipping row with duplicate invoice_id: INV-323
Skipping row with duplicate invoice_id: INV-324
Skipping row with duplicate invoice_id: INV-325
Skipping row with duplicate invoice_id: INV-326
Skipping row with duplicate invoice_id: INV-327
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-481
Skipping row with duplicate invoice_id: INV-482
Skipping row with duplicate invoice_id: INV-483
Skipping row with duplicate invoice_id: INV-484
Skipping row with duplicate invoice_id: INV-485
Skipping row with duplicate invoice_id: INV-486
Skipping row with duplicate invoice_id: INV-487
Skipping row with duplicate invoice_id: INV-488
Skipping row with duplicate invoice_id: INV-489
Skipping row with duplicate invoice_id: INV-490
Skipping row with duplicate invoice_id: INV-491
Skipping row with duplicate invoice_id: INV-492
Skipping row with duplicate invoice_id: INV-493
Skipping row with duplicate invoice_id: INV-494
Skipping row with duplicate invoice_id: INV-495
Skipping row with duplicate invoice_id: INV-496
Skipping row with duplicate invoice_id: INV-497
Skipping row with duplicate invoice_id: INV-498
Skipping row with duplicate invoice_id: INV-499
Skipping row with duplicate invoice_id: INV-500
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-698
Skipping row with duplicate invoice_id: INV-699
Skipping row with duplicate invoice_id: INV-700
Skipping row with duplicate invoice_id: INV-701
Skipping row with duplicate invoice_id: INV-702
Skipping row with duplicate invoice_id: INV-703
Skipping row with duplicate invoice_id: INV-704
Skipping row with duplicate invoice_id: INV-705
Skipping row with duplicate invoice_id: INV-706
Skipping row with duplicate invoice_id: INV-707
Skipping row with duplicate invoice_id: INV-708
Skipping row with duplicate invoice_id: INV-709
Skipping row with duplicate invoice_id: INV-710
Skipping row with duplicate invoice_id: INV-711
Skipping row with duplicate invoice_id: INV-712
Skipping row with duplicate invoice_id: INV-713
Skipping row with duplicate invoice_id: INV-714
Skipping row with duplicate invoice_id: INV-715
Skipping row with duplicate invoice_id: INV-716
Skipping row with duplicate invoice_id: INV-717
Skipping row with duplicate invoice_id: 

Skipping row with duplicate invoice_id: INV-895
Skipping row with duplicate invoice_id: INV-896
Skipping row with duplicate invoice_id: INV-897
Skipping row with duplicate invoice_id: INV-898
Skipping row with duplicate invoice_id: INV-899
Skipping row with duplicate invoice_id: INV-900
Skipping row with duplicate invoice_id: INV-901
Skipping row with duplicate invoice_id: INV-902
Skipping row with duplicate invoice_id: INV-903
Skipping row with duplicate invoice_id: INV-904
Skipping row with duplicate invoice_id: INV-905
Skipping row with duplicate invoice_id: INV-906
Skipping row with duplicate invoice_id: INV-907
Skipping row with duplicate invoice_id: INV-908
Skipping row with duplicate invoice_id: INV-909
Skipping row with duplicate invoice_id: INV-910
Skipping row with duplicate invoice_id: INV-911
Skipping row with duplicate invoice_id: INV-912
Skipping row with duplicate invoice_id: INV-913
Skipping row with duplicate invoice_id: INV-914
Skipping row with duplicate invoice_id: 

...

Skipping row with duplicate invoice_id: INV-21155
Skipping row with duplicate invoice_id: INV-21156
Skipping row with duplicate invoice_id: INV-21157
Skipping row with duplicate invoice_id: INV-21158
Skipping row with duplicate invoice_id: INV-21159
Skipping row with duplicate invoice_id: INV-21160
Skipping row with duplicate invoice_id: INV-21161
Skipping row with duplicate invoice_id: INV-21162
Skipping row with duplicate invoice_id: INV-21163
Skipping row with duplicate invoice_id: INV-21164
Skipping row with duplicate invoice_id: INV-21165
Skipping row with duplicate invoice_id: INV-21166
Skipping row with duplicate invoice_id: INV-21167
Skipping row with duplicate invoice_id: INV-21168
Skipping row with duplicate invoice_id: INV-21169
Skipping row with duplicate invoice_id: INV-21170
Skipping row with duplicate invoice_id: INV-21171
Skipping row with duplicate invoice_id: INV-21172
Skipping row with duplicate invoice_id: INV-21173
Skipping row with duplicate invoice_id: INV-21174


Skipping row with duplicate invoice_id: INV-21362
Skipping row with duplicate invoice_id: INV-21363
Skipping row with duplicate invoice_id: INV-21364
Skipping row with duplicate invoice_id: INV-21365
Skipping row with duplicate invoice_id: INV-21366
Skipping row with duplicate invoice_id: INV-21367
Skipping row with duplicate invoice_id: INV-21368
Skipping row with duplicate invoice_id: INV-21369
Skipping row with duplicate invoice_id: INV-21370
Skipping row with duplicate invoice_id: INV-21371
Skipping row with duplicate invoice_id: INV-21372
Skipping row with duplicate invoice_id: INV-21373
Skipping row with duplicate invoice_id: INV-21374
Skipping row with duplicate invoice_id: INV-21375
Skipping row with duplicate invoice_id: INV-21376
Skipping row with duplicate invoice_id: INV-21377
Skipping row with duplicate invoice_id: INV-21378
Skipping row with duplicate invoice_id: INV-21379
Skipping row with duplicate invoice_id: INV-21380
Skipping row with duplicate invoice_id: INV-21381


Skipping row with duplicate invoice_id: INV-21582
Skipping row with duplicate invoice_id: INV-21583
Skipping row with duplicate invoice_id: INV-21584
Skipping row with duplicate invoice_id: INV-21585
Skipping row with duplicate invoice_id: INV-21586
Skipping row with duplicate invoice_id: INV-21587
Skipping row with duplicate invoice_id: INV-21588
Skipping row with duplicate invoice_id: INV-21589
Skipping row with duplicate invoice_id: INV-21590
Skipping row with duplicate invoice_id: INV-21591
Skipping row with duplicate invoice_id: INV-21592
Skipping row with duplicate invoice_id: INV-21593
Skipping row with duplicate invoice_id: INV-21594
Skipping row with duplicate invoice_id: INV-21595
Skipping row with duplicate invoice_id: INV-21596
Skipping row with duplicate invoice_id: INV-21597
Skipping row with duplicate invoice_id: INV-21598
Skipping row with duplicate invoice_id: INV-21599
Skipping row with duplicate invoice_id: INV-21600
Skipping row with duplicate invoice_id: INV-21601


Skipping row with duplicate invoice_id: INV-21792
Skipping row with duplicate invoice_id: INV-21793
Skipping row with duplicate invoice_id: INV-21794
Skipping row with duplicate invoice_id: INV-21795
Skipping row with duplicate invoice_id: INV-21796
Skipping row with duplicate invoice_id: INV-21797
Skipping row with duplicate invoice_id: INV-21798
Skipping row with duplicate invoice_id: INV-21799
Skipping row with duplicate invoice_id: INV-21800
Skipping row with duplicate invoice_id: INV-21801
Skipping row with duplicate invoice_id: INV-21802
Skipping row with duplicate invoice_id: INV-21803
Skipping row with duplicate invoice_id: INV-21804
Skipping row with duplicate invoice_id: INV-21805
Skipping row with duplicate invoice_id: INV-21806
Skipping row with duplicate invoice_id: INV-21807
Skipping row with duplicate invoice_id: INV-21808
Skipping row with duplicate invoice_id: INV-21809
Skipping row with duplicate invoice_id: INV-21810
Skipping row with duplicate invoice_id: INV-21811


Skipping row with duplicate invoice_id: INV-21957
Skipping row with duplicate invoice_id: INV-21958
Skipping row with duplicate invoice_id: INV-21959
Skipping row with duplicate invoice_id: INV-21960
Skipping row with duplicate invoice_id: INV-21961
Skipping row with duplicate invoice_id: INV-21962
Skipping row with duplicate invoice_id: INV-21963
Skipping row with duplicate invoice_id: INV-21964
Skipping row with duplicate invoice_id: INV-21965
Skipping row with duplicate invoice_id: INV-21966
Skipping row with duplicate invoice_id: INV-21967
Skipping row with duplicate invoice_id: INV-21968
Skipping row with duplicate invoice_id: INV-21969
Skipping row with duplicate invoice_id: INV-21970
Skipping row with duplicate invoice_id: INV-21971
Skipping row with duplicate invoice_id: INV-21972
Skipping row with duplicate invoice_id: INV-21973
Skipping row with duplicate invoice_id: INV-21974
Skipping row with duplicate invoice_id: INV-21975
Skipping row with duplicate invoice_id: INV-21976


In [1]:
import pandas as pd

# Read the CSV file into a pandas dataframe
df = pd.read_csv('globbing.csv')

# Group the data by customer and count the number of invoices
grouped = df.groupby('Customer').agg({'InvoiceId': 'count'})

# Select only the customers with one invoice
result = grouped[grouped['InvoiceId'] == 1]

# Print the result
print(result)


           InvoiceId
Customer            
AAR366684          1
ATB262249          1
BRS840990          1
BSF439593          1
CKI107410          1
DAX787715          1
DFZ837701          1
DSE737276          1
EUF556994          1
GGL431742          1
GXL480310          1
HAH608808          1
IAY208308          1
IQN294014          1
IWU303142          1
JZG281834          1
KFT548996          1
KPO670477          1
KXH974893          1
LFG253534          1
NJM647149          1
OLY631292          1
RWD468759          1
SJZ770260          1
TNU309606          1
UCA347124          1
UYP958102          1
VXJ691842          1
XHB937113          1
XVA214225          1


In [1]:
from zenq.api import prepare_db

In [5]:
from zenq.visualizations import plot

In [6]:
from zenq import utils

In [7]:
plot.PlotUtils.pu()

ok
None


In [1]:
from zenq.api.prepare_db import db
m=db()
m.main()

Enter your database credentials
Username: aua
Password: mysecretpassword
Database name: GLOBBING
Initializing the database.. done


# Database

In [9]:
from zenq.logger import CustomFormatter, bcolors
import logging 

In [10]:
import os 
print(__name__)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(CustomFormatter())
logger.addHandler(ch)

logger.debug("debug message")
logger.info(f"{bcolors.OKGREEN} Warning: Email has not been sent......{bcolors.OKGREEN}")
logger.warning("warning message")
logger.error("error message")
logger.critical("critical message")

2023-04-21 20:19:38,503 - __main__ - DEBUG - debug message (789279774.py:10)
2023-04-21 20:19:38,505 - __main__ - INFO -  Warning: Email has not been sent...... (789279774.py:11)
2023-04-21 20:19:38,506 - __main__ - WARNING - warning message (789279774.py:12)
2023-04-21 20:19:38,507 - __main__ - ERROR - error message (789279774.py:13)
2023-04-21 20:19:38,508 - __main__ - CRITICAL - critical message (789279774.py:14)


__main__


In [1]:
from zenq.api.endpoints import points
points_instance = points('globbing.csv')
# points_instance.insert_facts()

points_instance.top_10_customer_expected_purchase_in_30_days()



Enter your database credentials
Username: aua
Password: mysecretpassword
Database name: GLOBBING


,customer_id,expected_purchase
0,ALN161735,3.230200
1,RFS781349,2.781714
2,QYV196131,2.689526
3,AZP596115,2.662708
4,SCK314236,2.659689
5,ZGV934805,2.657912
6,ZFZ316415,2.648490
7,LPQ959075,2.647007
8,HPI478895,2.614142
9,ZJF744789,2.612179


In [1]:
from zenq.clvmodels.pareto import Model
model = Model()

result = model.cltv_df()
# result = model.rfm()


result


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %


,customer_id,min_date,recency,T,frequency,monetary
0,KVO444312,2022-09-13,163,229,14,255590.0
1,VBV804469,2022-09-04,200,238,14,310880.0
2,BAE240832,2022-09-13,196,229,3,39005.0
3,AOK457989,2022-09-14,189,228,5,111895.0
4,XZF813575,2022-09-05,179,237,11,223440.0
...,...,...,...,...,...,...
1956,DXT747575,2022-09-12,206,230,7,114810.0
1957,VOZ592840,2023-01-14,52,106,2,51295.0
1958,ZCQ782298,2022-09-13,207,229,20,465100.0
1959,YNW866090,2022-09-02,143,240,11,238605.0


In [3]:
from lifetimes import ParetoNBDFitter

from zenq.clvmodels.pareto import Model
model = Model()
result = model.fit_paretonbd()
# print('Length of df_data_group:', len(result))
freq=result.params_ 
freq

Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


r             7.169469
alpha       132.034107
s           359.437591
beta     377710.951537
dtype: float64

In [ ]:

from lifetimes import ParetoNBDFitter
from lifetimes import ParetoNBDFitter 
from lifetimes.utils import summary_data_from_transaction_data
import pandas as pd 
from zenq.clvmodels.pareto import Model
model = Model()

result = model.fit_paretonbd()

# print('Length of df_data_group:', len(result))

result

In [1]:

from lifetimes import ParetoNBDFitter
from lifetimes import ParetoNBDFitter 
from lifetimes.utils import summary_data_from_transaction_data
import pandas as pd 
from zenq.clvmodels.pareto import Model
uf = Model()

result = uf.rfm()

# print('Length of df_data_group:', len(result))

result

Enter your database credentials
Username: aua
Password: mysecretpassword
Database name: GLOBBING
Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  227         14  255590.0
1      VBV804469 2022-09-04      200  236         14  310880.0
2      BAE240832 2022-09-13      196  227          3   39005.0
3      AOK457989 2022-09-14      189  226          5  111895.0
4      XZF813575 2022-09-05      179  235         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  228          7  114810.0
1957   VOZ592840 2023-01-14       52  104          2   51295.0
1958   ZCQ782298 2022-09-13      207  227         20  465100.0
1959   YNW866090 2022-09-02      143  238         11  238605.0
1960   IFC198488 2022-09-17      189  223          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  227         14  255590.0
1      VBV804469 2022-09-04      200  236         14  310880.0
2      BAE240832 2022-09-13      196  227          3   39005.0
3      AOK457989 2022-09-14      189  226          5  111895.0
4      XZF813575 2022-09-05      179  235         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  228          7  114810.0
1957   VOZ592840 2023-01-14       52  104          2   51295.0
1958   ZCQ782298 2022-09-13      207  227         20  465100.0
1959   YNW866090 2022-09-02      143  238         11  238605.0
1960   IFC198488 2022-09-17      189  223          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  227         14  255590.0
1      VBV804469 2022-09-04      200  236         14  310880.0
2      BAE240832 2022-09-13      196  227          3   39005.0
3      AOK457989 2022-09-14      189  226          5  111895.0
4      XZF813575 2022-09-05      179  235         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  228          7  114810.0
1957   VOZ592840 2023-01-14       52  104          2   51295.0
1958   ZCQ782298 2022-09-13      207  227         20  465100.0
1959   YNW866090 2022-09-02      143  238         11  238605.0
1960   IFC198488 2022-09-17      189  223          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %


,customer_id,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,KVO444312,4,4,4,44,LOYAL CUSTOMER
1,VBV804469,2,4,4,24,AT RISK
2,BAE240832,2,1,1,21,HIBERNATING
3,AOK457989,3,1,2,31,ABOUT TO SLEEP
4,XZF813575,3,3,3,33,NEED ATTENTION
...,...,...,...,...,...,...
1956,DXT747575,1,2,2,12,HIBERNATING
1957,VOZ592840,5,1,1,51,NEW CUSTOMERS
1958,ZCQ782298,1,5,5,15,CANT LOSE
1959,YNW866090,4,3,3,43,POTENTIAL LOYALIST


In [3]:

from lifetimes import ParetoNBDFitter
from lifetimes import ParetoNBDFitter 
from lifetimes.utils import summary_data_from_transaction_data
import pandas as pd 
from zenq.clvmodels.pareto import Model
uf = Model()

result = uf.cltv_df()

# print('Length of df_data_group:', len(result))

result
 

Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %
     customer_id   min_date  recency    T  frequency  monetary
0      KVO444312 2022-09-13      163  229         14  255590.0
1      VBV804469 2022-09-04      200  238         14  310880.0
2      BAE240832 2022-09-13      196  229          3   39005.0
3      AOK457989 2022-09-14      189  228          5  111895.0
4      XZF813575 2022-09-05      179  237         11  223440.0
...          ...        ...      ...  ...        ...       ...
1956   DXT747575 2022-09-12      206  230          7  114810.0
1957   VOZ592840 2023-01-14       52  106          2   51295.0
1958   ZCQ782298 2022-09-13      207  229         20  465100.0
1959   YNW866090 2022-09-02      143  240         11  238605.0
1960   IFC198488 2022-09-17      189  225          6  174255.0

[1960 rows x 6 columns]


/opt/anaconda3/lib/python3.8/site-packages/scipy/special/_logsumexp.py:108: RuntimeWarning: overflow encountered in exp
  tmp = b * np.exp(a - a_max)


Percentage of customers that only bought once 0.0 %
Percentage of customers that only bought once 0.0 %


,customer_id,min_date,recency,T,frequency,monetary
0,KVO444312,2022-09-13,163,229,14,255590.0
1,VBV804469,2022-09-04,200,238,14,310880.0
2,BAE240832,2022-09-13,196,229,3,39005.0
3,AOK457989,2022-09-14,189,228,5,111895.0
4,XZF813575,2022-09-05,179,237,11,223440.0
...,...,...,...,...,...,...
1956,DXT747575,2022-09-12,206,230,7,114810.0
1957,VOZ592840,2023-01-14,52,106,2,51295.0
1958,ZCQ782298,2022-09-13,207,229,20,465100.0
1959,YNW866090,2022-09-02,143,240,11,238605.0


In [7]:
from zenq.visualizations.plot import Visuals
uf = Visuals()

# result = uf.total_sales_by_location()


# result = uf.price_distribution()
# result = uf.time_series()
# result = uf.gender_price()
# result = uf.rfm_treemap()
result = uf.top_customers_90days()


# result = uf.customer_aliveness()
# result = uf.customer_aliveness()


result
